![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [216]:
import pandas as pd
pd.set_option('display.max_columns', None) 
import numpy as np

# Start coding here...

In [217]:
df = pd.read_csv("bank_marketing.csv")
#examining the dataframe
print(df.head())
print(df.info())
print(df.shape)

   client_id  age        job  marital    education credit_default mortgage  \
0          0   56  housemaid  married     basic.4y             no       no   
1          1   57   services  married  high.school        unknown       no   
2          2   37   services  married  high.school             no      yes   
3          3   40     admin.  married     basic.6y             no       no   
4          4   56   services  married  high.school             no       no   

  month  day  contact_duration  number_contacts  previous_campaign_contacts  \
0   may   13               261                1                           0   
1   may   19               149                1                           0   
2   may   23               226                1                           0   
3   may   27               151                1                           0   
4   may    3               307                1                           0   

  previous_outcome  cons_price_idx  euribor_three_months

In [218]:
#cleaning the job column
#print(df['job'].value_counts())
#replacing the . with _
df['job'] = df['job'].str.replace('.','_')
#examining changes
print(df['job'].value_counts())

admin_           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64


In [219]:
#cleaning the education column
#print(df['education'].value_counts())
#replacing the . with _
df['education'] = df['education'].str.replace('.','_')
df['education'] = df['education'].replace('unknown', np.NaN)
#examining changes
print(df['education'].value_counts())

university_degree      12168
high_school             9515
basic_9y                6045
professional_course     5243
basic_4y                4176
basic_6y                2292
illiterate                18
Name: education, dtype: int64


In [220]:
#examining the credit default column
print(df['credit_default'].info())
#converting credit default values to 1 and 0
df['credit_default'] = df['credit_default'].map({'yes':1,'no':0, 'unknown':0})
df['credit_default'] = df['credit_default'].astype('bool')
#examining changes
print(f"Final Data Type: {df['credit_default'].dtype}")

<class 'pandas.core.series.Series'>
RangeIndex: 41188 entries, 0 to 41187
Series name: credit_default
Non-Null Count  Dtype 
--------------  ----- 
41188 non-null  object
dtypes: object(1)
memory usage: 321.9+ KB
None
Final Data Type: bool


In [221]:
#examining the mortage column
print(df['mortgage'].value_counts())
#converting credit default values to 1 and 0
df['mortgage'] = df['mortgage'].map({'yes':1,'no':0, 'unknown':0})
df['mortgage'] = df['mortgage'].astype(bool)
#examining changes
print(f"Final Data Type: {df['mortgage'].dtype}")

yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
Final Data Type: bool


In [222]:
#creating client_df
client_df = df[['client_id','age','job','marital','education','credit_default','mortgage']]
print(client_df.head())
#creating client.csv
client_df.to_csv('client.csv',index=False)

   client_id  age        job  marital    education  credit_default  mortgage
0          0   56  housemaid  married     basic_4y           False     False
1          1   57   services  married  high_school           False     False
2          2   37   services  married  high_school           False      True
3          3   40     admin_  married     basic_6y           False     False
4          4   56   services  married  high_school           False     False


In [223]:
#exploring the previous_outcome column
#print(df['previous_outcome'].value_counts())
#converting previous_outcome values to 1 and 0
df['previous_outcome'] = df['previous_outcome'].map({'success':1,'failure':0, 'nonexistent':0})
df['previous_outcome'] = df['previous_outcome'].astype(bool)
#examining changes
print(df['previous_outcome'].value_counts())
print(f"Final Data Type: {df['previous_outcome'].dtype}")

False    39815
True      1373
Name: previous_outcome, dtype: int64
Final Data Type: bool


In [224]:
#exploring the campaign_outcome column
print(df['campaign_outcome'].value_counts())
#converting campaign_outcome values to 1 and 0
df['campaign_outcome'] = df['campaign_outcome'].map({'no':0,'yes':1})
df['campaign_outcome'] = df['campaign_outcome'].astype(bool)
#examining changes
print(df['campaign_outcome'].value_counts())
print(f"Final Data Type: {df['campaign_outcome'].dtype}")

no     36548
yes     4640
Name: campaign_outcome, dtype: int64
False    36548
True      4640
Name: campaign_outcome, dtype: int64
Final Data Type: bool


In [225]:
#creating the last_contact_date column
#creating the year column
df['year'] = 2022
#converting the month column to string
df['year'] = df['year'].astype(str)
print(f"Final data type of year column: {df['year'].dtype}")
#examining the month column
print(df['month'].value_counts())
#mapping the month values to numbers
df['month'] = df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12})
#converting the month column to string
df['month'] = df['month'].astype(str)
#examining changes
print(f"Final data type of month column: {df['month'].dtype}")
#creating last_contact_date column
df['date_string'] = df['year'] + '-' + df['month']+ '-' + df['day'].astype(str)
#converting the column to a datetime object
df['last_contact_date'] = pd.to_datetime(df['date_string'], format='%Y-%m-%d')
#examining the result
print(df['last_contact_date'].value_counts())
print(f"Final data type of last_contact_date column: {df['last_contact_date'].dtype}")

Final data type of year column: object
may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: month, dtype: int64
Final data type of month column: object
2022-05-22    497
2022-05-07    493
2022-05-01    479
2022-05-05    473
2022-05-25    465
             ... 
2022-12-27      4
2022-12-26      4
2022-12-03      4
2022-12-15      3
2022-12-19      3
Name: last_contact_date, Length: 306, dtype: int64
Final data type of last_contact_date column: datetime64[ns]


In [226]:
#creating the campaign.csv file 
campaign_df = df[['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome','last_contact_date']]
#examining campaign_df
print(campaign_df.head())
#creating the campaign_csv
campaign_df.to_csv('campaign.csv',index=False)


   client_id  number_contacts  contact_duration  previous_campaign_contacts  \
0          0                1               261                           0   
1          1                1               149                           0   
2          2                1               226                           0   
3          3                1               151                           0   
4          4                1               307                           0   

   previous_outcome  campaign_outcome last_contact_date  
0             False             False        2022-05-13  
1             False             False        2022-05-19  
2             False             False        2022-05-23  
3             False             False        2022-05-27  
4             False             False        2022-05-03  


In [227]:
#creating economics.csv file
economics_df = df[['client_id','cons_price_idx','euribor_three_months']]
print(economics_df.head())
economics_df.to_csv('economics.csv',index=False)

   client_id  cons_price_idx  euribor_three_months
0          0          93.994                 4.857
1          1          93.994                 4.857
2          2          93.994                 4.857
3          3          93.994                 4.857
4          4          93.994                 4.857
